In [ ]:
def display_image(img):
    display(Image.fromarray(img))

In [1]:
import torchvision.references.transforms as my_transforms
def get_transform(train):
    abc = []
    abc.append(my_transforms.ToTensor())
    if train:
        abc.append(my_transforms.RandomHorizontalFlip(0.5))
    return my_transforms.Compose(abc)

In [2]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from PIL import Image
from PIL import Image, ImageDraw
import requests
from pycocotools.coco import COCO
from pathlib import Path
import cv2
import torch
import os
import sys
from torchvision import transforms
import mrcnn

# Set the ROOT_DIR variable to the root directory of the Mask_RCNN git repo
ROOT_DIR = './Mask_RCNN'
assert os.path.exists(ROOT_DIR), 'ROOT_DIR does not exist. Did you forget to read the instructions above? ;)'

# Import mrcnn libraries
sys.path.append(ROOT_DIR) 
from mrcnn.config import Config
# import mrcnn.utils as utils
from mrcnn import visualize
# import mrcnn.model as modellib
from torchvision.ops import box_convert

In [3]:
class LiveCellDataset(torch.utils.data.Dataset): #torch.utils.data.Dataset
    def __init__(self, root, transforms):
        self.root = root
        self.transforms = transforms
        
        # Loading annotations here
        #self.coco_annotation = COCO(annotation_file="../data/TrashCan/instances_train_trashcan.json")
        self.coco_annotation = COCO(annotation_file="../data/TrashCan/instances_val_trashcan.json")
        #self.coco_annotation = COCO(annotation_file="../data/LiveCell/livecell_coco_train.json")
        
        # Loading their ids, please note only one category!
        self.image_ids = self.coco_annotation.getImgIds(catIds=[self.coco_annotation.getCatIds()[0]])

    def __getitem__(self, idx):
        img_id = self.image_ids[idx]
        img_info = self.coco_annotation.loadImgs([img_id])[0]
        img_file_name = img_info["file_name"]
        
        #img_path = Path("../data/TrashCan/train") / img_file_name
        img_path = Path("../data/TrashCan/val") / img_file_name
#         splitted = img_file_name.split("_")
#         img_path = Path("../data/LiveCell/images/livecell_train_val_images/") / splitted[0] / img_file_name
        
        img = Image.open(str(img_path)).convert("RGB")
        #img = np.asarray(img) #/ 255
#         img = torch.as_tensor(np.copy(img), dtype=torch.float64)
#         display_image(img)
        
        ann_ids = self.coco_annotation.getAnnIds(imgIds=[img_id], iscrowd=None)
        anns = self.coco_annotation.loadAnns(ann_ids)
        
        target = self.get_target(anns, img_info, idx)
        
#         #----
#         masks2, class_ids = self.get_masks(anns, img_info)
#         masks2 = torch.as_tensor(np.copy(masks2), dtype=torch.uint8)
# #         target['class_ids'] = class_ids
#         target['masks'] = masks2
#         class_ids = torch.as_tensor(class_ids, dtype=torch.int64)
#         target['labels'] = class_ids
#         cat_names = np.array(['cell' for x in range(0, len(class_ids))])
# #         target['cat_names'] = cat_names
#         #----
    
    
        if self.transforms is not None:
            img, target = self.transforms(img, target)
        
#         return transforms.ToTensor()(img), target

        return img, target

    def get_masks(self, annotations, image_info):
        instance_masks = []
        class_ids = []
        
        for annotation in annotations:
            class_id = annotation['category_id']
            mask = Image.new('1', (image_info['width'], image_info['height']))
            mask_draw = ImageDraw.ImageDraw(mask, '1')
            for segmentation in annotation['segmentation']:
                mask_draw.polygon(segmentation, fill=1)
                bool_array = np.array(mask) > 0
                instance_masks.append(bool_array)
                class_ids.append(class_id)

        mask = np.dstack(instance_masks)
        
        class_ids = np.array(class_ids, dtype=np.int32)
        
        return mask, class_ids
    
    def get_target(self, annotations, image_info, idx):
#         print(annotations, len(annotations))
        boxes = []
        labels = []
        masks = []
        area = []
        iscrowd = []
        
        image_id = torch.tensor([idx], dtype=torch.int64)
        
        for annotation in annotations:
#             print(len(annotation['segmentation']))
            one_mask = self.coco_annotation.annToMask(annotation)
            masks.append(one_mask)
            
            labels.append(annotation['category_id'])
            
            bounding_box = annotation['bbox']
            boxes.append(bounding_box)
            
            _iscrowd = annotation['iscrowd']
            iscrowd.append(_iscrowd)
            
            _area = annotation['area']
            area.append(_area)
            
#         masks = transforms.ToTensor()(np.dstack(masks))
        masks = np.dstack(masks)
        masks = torch.as_tensor(np.copy(masks), dtype=torch.uint8)
        masks = torch.permute(masks, (2,0,1))
        
        # THERE IS A PROBLEM WITH LABELS!!!!!!!!!!!!!!!!!!!!!!!!!!!! TODO
#         labels.sort()
        labels = torch.as_tensor(labels, dtype=torch.int64)
#         print(torch.ones((len(labels),), dtype=torch.int64))
#         labels = torch.ones((len(labels),), dtype=torch.int64)
    
        area = torch.as_tensor(area, dtype=torch.float64)
        iscrowd = torch.as_tensor(iscrowd, dtype=torch.uint8)
        
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        boxes = box_convert(boxes, in_fmt='xywh', out_fmt='xyxy')
        
        assert len(boxes) == len(iscrowd) == len(labels) == len(area)
        if not (len(boxes) == len(iscrowd) == len(labels) == len(area)):
            print(len(boxes), len(iscrowd), len(labels), len(area))
        
        
        target = {}
        target["boxes"] = boxes

        #---
        target["labels"] = labels
        target["masks"] = masks
        #---
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd
        return target

    def __len__(self):
        return len(self.image_ids)
dataset = LiveCellDataset("LiveCellDataset", get_transform(train=True))

loading annotations into memory...
Done (t=0.05s)
creating index...
index created!


In [4]:
image, target = dataset.__getitem__(15)
print(image.shape)
print(target['masks'].shape)
print('labels', target['labels'])
print('boxes:', target['boxes'].shape) #correct
print('iscrowd:', target['iscrowd'])
print('image_id:', target['image_id'])
print('area:', target['area'])
# print(target['masks'])
# mask = target['masks']
# print(np.sum(mask[:, :, np.where(target['labels'] == 1)[0]]))
# masks = target['masks']
# torch.permute(masks, (2,0,1)).size()
# masks = np.asarray(target['masks'])
# print(masks.shape)
# print(np.flip(masks,1).shape)
# print(np.flip(masks,0).shape)
# print(masks[::-1].shape)
# print(masks[:,:,::-1].shape)
# print(np.moveaxis(masks, -1, 0).shape)


torch.Size([3, 270, 480])
torch.Size([3, 270, 480])
labels tensor([ 1,  1, 14])
boxes: torch.Size([3, 4])
iscrowd: tensor([0, 0, 0], dtype=torch.uint8)
image_id: tensor([15])
area: tensor([26386.7049, 24395.7726,  2909.6177], dtype=torch.float64)


In [ ]:
image_ids = np.random.choice(list(range(0, dataset.__len__())), 4)
for image_id in image_ids:
    image, target = dataset.__getitem__(image_id)
    print('image', image.shape)
    print('masks', target['masks'].shape)
    print('labels', target['labels'])
    print('boxes:', target['boxes'].shape) #correct
    print('iscrowd:', target['iscrowd'])
    print('image_id:', target['image_id'])
    print('area:', target['area'])
    print(target['masks2'].shape)
    print(target['masks'].shape)
    visualize.display_top_masks(image, target['masks2'], target['class_ids'], np.array(["a" + str(_) for _ in range(22)]))

In [ ]:
for x in range(0, dataset.__len__()):
    image, target = dataset.__getitem__(x)
    print('image', image.shape)
    print('masks', target['masks'].shape)
    print('labels', target['labels'])
    print('boxes:', target['boxes'].shape) #correct
    print('iscrowd:', target['iscrowd'])
    print('image_id:', target['image_id'])
    print('area:', target['area'])
    print()
    if x == 50:
        break

In [ ]:
# model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
# dataset = LiveCellDataset("LiveCellDataset", None)
# data_loader = torch.utils.data.DataLoader(
#  dataset, batch_size=2, shuffle=True, num_workers=4,
#  collate_fn=collate_fn)
# # For Training
# images,targets = next(iter(data_loader))
# images = list(image for image in images)
# targets = [{k: v for k, v in t.items()} for t in targets]
# output = model(images,targets)   # Returns losses and detections
# # For inference
# model.eval()
# x = [torch.rand(3, 300, 400), torch.rand(3, 500, 400)]
# predictions = model(x)           # Returns predictions


In [5]:
def collate_fn(batch):
    return tuple(zip(*batch))

data_loader = torch.utils.data.DataLoader(
dataset, batch_size=1, shuffle=True, num_workers=4, collate_fn=collate_fn)

import torchvision
from torchvision.references.engine import train_one_epoch, evaluate
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor

def get_model_instance_segmentation(num_classes):
    # load an instance segmentation model pre-trained on COCO
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)

    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # now get the number of input features for the mask classifier
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    # and replace the mask predictor with a new one
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
                                                       hidden_layer,
                                                       num_classes)

    return model

def main():
    # train on the GPU or on the CPU, if a GPU is not available
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

    # our dataset has two classes only - background and person
    # 1 + 22
    num_classes = 23

    # get the model using our helper function
    model = get_model_instance_segmentation(num_classes)

    # move model to the right device
    model.to(device)

    # construct an optimizer
    params = [p for p in model.parameters() if p.requires_grad]
    optimizer = torch.optim.SGD(params, lr=0.005,
                                momentum=0.9, weight_decay=0.0005)
    # and a learning rate scheduler
    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                                   step_size=3,
                                                   gamma=0.1)
    # let's train it for 10 epochs
    num_epochs = 4

    for epoch in range(num_epochs):
        # train for one epoch, printing every 10 iterations
        train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
        # update the learning rate
        lr_scheduler.step()
        # evaluate on the test dataset
        evaluate(model, data_loader, device=device)
    
    return model 
#     print("That's it!")
model = main()

/home/janek/anaconda3/envs/yolo6/lib/python3.8/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1639180544224/work/aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Epoch: [0]  [  0/524]  eta: 0:07:35  lr: 0.000015  loss: 5.4483 (5.4483)  loss_classifier: 3.7977 (3.7977)  loss_box_reg: 0.2461 (0.2461)  loss_mask: 1.2343 (1.2343)  loss_objectness: 0.1569 (0.1569)  loss_rpn_box_reg: 0.0133 (0.0133)  time: 0.8700  data: 0.1465  max mem: 1379
Epoch: [0]  [ 10/524]  eta: 0:04:00  lr: 0.000110  loss: 4.9952 (4.8988)  loss_classifier: 3.5507 (3.4848)  loss_box_reg: 0.2577 (0.2565)  loss_mask: 1.0135 (1.0740)  loss_objectness: 0.0564 (0.0624)  loss_rpn_box_reg: 0.0073 (0.0212)  time: 0.4687  data: 0.0144  max mem: 1650
Epoch: [0]  [ 20/524]  eta: 0:03:47  lr: 0.000206  loss: 4.0866 (3.8558)  loss_classifier: 2.7192 (2.5385)  loss_box_reg: 0.2577 (0.2509)  loss_mask: 0.9520 (0.9893)  loss_objectness: 0.0403 (0.0500)  loss_rpn_box_reg: 0.0086 (0.0270)  time: 0.4309  data: 0.0014  max mem: 1650
Epoch: [0]  [ 30/524]  eta: 0:03:39  lr: 0.000301  loss: 1.6050 (3.0372)  loss_classifier: 0.5724 (1.8520)  loss_box_reg: 0.2410 (0.2455)  loss_mask: 0.7611 (0.8770) 

Epoch: [0]  [300/524]  eta: 0:01:42  lr: 0.002880  loss: 0.8140 (1.2544)  loss_classifier: 0.1938 (0.4657)  loss_box_reg: 0.1938 (0.2679)  loss_mask: 0.2837 (0.4570)  loss_objectness: 0.0106 (0.0400)  loss_rpn_box_reg: 0.0107 (0.0237)  time: 0.4642  data: 0.0019  max mem: 1694
Epoch: [0]  [310/524]  eta: 0:01:37  lr: 0.002975  loss: 0.9051 (1.2439)  loss_classifier: 0.2678 (0.4596)  loss_box_reg: 0.2421 (0.2666)  loss_mask: 0.3662 (0.4539)  loss_objectness: 0.0106 (0.0393)  loss_rpn_box_reg: 0.0196 (0.0245)  time: 0.4637  data: 0.0019  max mem: 1694
Epoch: [0]  [320/524]  eta: 0:01:33  lr: 0.003071  loss: 0.7521 (1.2255)  loss_classifier: 0.1741 (0.4510)  loss_box_reg: 0.1655 (0.2641)  loss_mask: 0.2541 (0.4477)  loss_objectness: 0.0080 (0.0384)  loss_rpn_box_reg: 0.0147 (0.0242)  time: 0.4635  data: 0.0019  max mem: 1694
Epoch: [0]  [330/524]  eta: 0:01:28  lr: 0.003166  loss: 0.5811 (1.2116)  loss_classifier: 0.1559 (0.4442)  loss_box_reg: 0.1457 (0.2623)  loss_mask: 0.2498 (0.4430) 

Epoch: [1]  [  0/524]  eta: 0:05:01  lr: 0.005000  loss: 0.8899 (0.8899)  loss_classifier: 0.2494 (0.2494)  loss_box_reg: 0.1889 (0.1889)  loss_mask: 0.3822 (0.3822)  loss_objectness: 0.0140 (0.0140)  loss_rpn_box_reg: 0.0554 (0.0554)  time: 0.5749  data: 0.1082  max mem: 1694
Epoch: [1]  [ 10/524]  eta: 0:03:54  lr: 0.005000  loss: 0.8488 (0.8076)  loss_classifier: 0.2494 (0.2663)  loss_box_reg: 0.1597 (0.1937)  loss_mask: 0.3025 (0.2985)  loss_objectness: 0.0134 (0.0205)  loss_rpn_box_reg: 0.0149 (0.0285)  time: 0.4571  data: 0.0113  max mem: 1694
Epoch: [1]  [ 20/524]  eta: 0:03:49  lr: 0.005000  loss: 0.7584 (0.8605)  loss_classifier: 0.2525 (0.2775)  loss_box_reg: 0.2030 (0.2237)  loss_mask: 0.3025 (0.3004)  loss_objectness: 0.0161 (0.0336)  loss_rpn_box_reg: 0.0140 (0.0253)  time: 0.4502  data: 0.0018  max mem: 1694
Epoch: [1]  [ 30/524]  eta: 0:03:43  lr: 0.005000  loss: 0.7154 (0.8156)  loss_classifier: 0.2525 (0.2531)  loss_box_reg: 0.2030 (0.2074)  loss_mask: 0.3036 (0.3025) 

Epoch: [1]  [300/524]  eta: 0:01:41  lr: 0.005000  loss: 0.6014 (0.7654)  loss_classifier: 0.1156 (0.2323)  loss_box_reg: 0.1077 (0.2007)  loss_mask: 0.2275 (0.2803)  loss_objectness: 0.0117 (0.0248)  loss_rpn_box_reg: 0.0110 (0.0273)  time: 0.4421  data: 0.0019  max mem: 1694
Epoch: [1]  [310/524]  eta: 0:01:36  lr: 0.005000  loss: 0.5577 (0.7614)  loss_classifier: 0.0985 (0.2307)  loss_box_reg: 0.1291 (0.1998)  loss_mask: 0.2070 (0.2792)  loss_objectness: 0.0078 (0.0244)  loss_rpn_box_reg: 0.0090 (0.0273)  time: 0.4370  data: 0.0019  max mem: 1694
Epoch: [1]  [320/524]  eta: 0:01:31  lr: 0.005000  loss: 0.5992 (0.7570)  loss_classifier: 0.1414 (0.2282)  loss_box_reg: 0.1570 (0.1990)  loss_mask: 0.2251 (0.2786)  loss_objectness: 0.0058 (0.0241)  loss_rpn_box_reg: 0.0133 (0.0272)  time: 0.4485  data: 0.0019  max mem: 1694
Epoch: [1]  [330/524]  eta: 0:01:27  lr: 0.005000  loss: 0.5261 (0.7563)  loss_classifier: 0.1369 (0.2278)  loss_box_reg: 0.1570 (0.1989)  loss_mask: 0.2311 (0.2778) 

Epoch: [2]  [  0/524]  eta: 0:04:55  lr: 0.005000  loss: 0.5481 (0.5481)  loss_classifier: 0.1610 (0.1610)  loss_box_reg: 0.1285 (0.1285)  loss_mask: 0.2320 (0.2320)  loss_objectness: 0.0220 (0.0220)  loss_rpn_box_reg: 0.0047 (0.0047)  time: 0.5647  data: 0.1191  max mem: 1694
Epoch: [2]  [ 10/524]  eta: 0:03:49  lr: 0.005000  loss: 0.5481 (0.6572)  loss_classifier: 0.1610 (0.1960)  loss_box_reg: 0.1621 (0.1478)  loss_mask: 0.2320 (0.2474)  loss_objectness: 0.0167 (0.0284)  loss_rpn_box_reg: 0.0158 (0.0378)  time: 0.4459  data: 0.0125  max mem: 1694
Epoch: [2]  [ 20/524]  eta: 0:03:44  lr: 0.005000  loss: 0.6151 (0.6496)  loss_classifier: 0.1474 (0.1844)  loss_box_reg: 0.1445 (0.1526)  loss_mask: 0.2421 (0.2549)  loss_objectness: 0.0167 (0.0242)  loss_rpn_box_reg: 0.0158 (0.0335)  time: 0.4404  data: 0.0018  max mem: 1694
Epoch: [2]  [ 30/524]  eta: 0:03:40  lr: 0.005000  loss: 0.6246 (0.6404)  loss_classifier: 0.1366 (0.1779)  loss_box_reg: 0.1445 (0.1580)  loss_mask: 0.2630 (0.2507) 

Epoch: [2]  [300/524]  eta: 0:01:42  lr: 0.005000  loss: 0.6705 (0.6580)  loss_classifier: 0.1732 (0.1821)  loss_box_reg: 0.1747 (0.1820)  loss_mask: 0.2242 (0.2473)  loss_objectness: 0.0098 (0.0168)  loss_rpn_box_reg: 0.0185 (0.0298)  time: 0.4654  data: 0.0019  max mem: 1694
Epoch: [2]  [310/524]  eta: 0:01:38  lr: 0.005000  loss: 0.5118 (0.6547)  loss_classifier: 0.1172 (0.1806)  loss_box_reg: 0.1717 (0.1819)  loss_mask: 0.2214 (0.2461)  loss_objectness: 0.0093 (0.0168)  loss_rpn_box_reg: 0.0150 (0.0293)  time: 0.4643  data: 0.0020  max mem: 1694
Epoch: [2]  [320/524]  eta: 0:01:33  lr: 0.005000  loss: 0.4270 (0.6538)  loss_classifier: 0.0875 (0.1800)  loss_box_reg: 0.1087 (0.1824)  loss_mask: 0.1915 (0.2458)  loss_objectness: 0.0052 (0.0167)  loss_rpn_box_reg: 0.0087 (0.0290)  time: 0.4684  data: 0.0020  max mem: 1694
Epoch: [2]  [330/524]  eta: 0:01:28  lr: 0.005000  loss: 0.5045 (0.6508)  loss_classifier: 0.0984 (0.1789)  loss_box_reg: 0.1609 (0.1819)  loss_mask: 0.1994 (0.2448) 

 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.103
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.062
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.160
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.193
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.194
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.216
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.184
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.210
IoU metric: segm
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.077
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.148
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.061
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.105
 Average Precision  (AP

Epoch: [3]  [240/524]  eta: 0:02:11  lr: 0.000500  loss: 0.3512 (0.5244)  loss_classifier: 0.0541 (0.1356)  loss_box_reg: 0.0961 (0.1549)  loss_mask: 0.1648 (0.2029)  loss_objectness: 0.0034 (0.0105)  loss_rpn_box_reg: 0.0044 (0.0205)  time: 0.4724  data: 0.0019  max mem: 1694
Epoch: [3]  [250/524]  eta: 0:02:06  lr: 0.000500  loss: 0.3028 (0.5203)  loss_classifier: 0.0511 (0.1356)  loss_box_reg: 0.0776 (0.1533)  loss_mask: 0.1529 (0.2010)  loss_objectness: 0.0031 (0.0104)  loss_rpn_box_reg: 0.0043 (0.0200)  time: 0.4705  data: 0.0019  max mem: 1694
Epoch: [3]  [260/524]  eta: 0:02:02  lr: 0.000500  loss: 0.3542 (0.5185)  loss_classifier: 0.0665 (0.1345)  loss_box_reg: 0.0824 (0.1536)  loss_mask: 0.1448 (0.2004)  loss_objectness: 0.0027 (0.0101)  loss_rpn_box_reg: 0.0056 (0.0198)  time: 0.4712  data: 0.0020  max mem: 1694
Epoch: [3]  [270/524]  eta: 0:01:57  lr: 0.000500  loss: 0.4267 (0.5228)  loss_classifier: 0.0893 (0.1349)  loss_box_reg: 0.1152 (0.1562)  loss_mask: 0.1781 (0.2012) 

creating index...
index created!
Test:  [  0/524]  eta: 0:02:28  model_time: 0.1665 (0.1665)  evaluator_time: 0.0046 (0.0046)  time: 0.2837  data: 0.1117  max mem: 1694
Test:  [100/524]  eta: 0:01:15  model_time: 0.1691 (0.1682)  evaluator_time: 0.0065 (0.0061)  time: 0.1822  data: 0.0020  max mem: 1694
Test:  [200/524]  eta: 0:00:57  model_time: 0.1648 (0.1689)  evaluator_time: 0.0056 (0.0065)  time: 0.1745  data: 0.0019  max mem: 1694
Test:  [300/524]  eta: 0:00:40  model_time: 0.1672 (0.1692)  evaluator_time: 0.0047 (0.0067)  time: 0.1769  data: 0.0018  max mem: 1694
Test:  [400/524]  eta: 0:00:22  model_time: 0.1661 (0.1695)  evaluator_time: 0.0064 (0.0067)  time: 0.1791  data: 0.0018  max mem: 1694
Test:  [500/524]  eta: 0:00:04  model_time: 0.1688 (0.1692)  evaluator_time: 0.0050 (0.0067)  time: 0.1794  data: 0.0018  max mem: 1694
Test:  [523/524]  eta: 0:00:00  model_time: 0.1656 (0.1691)  evaluator_time: 0.0051 (0.0067)  time: 0.1739  data: 0.0018  max mem: 1694
Test: Total tim

In [7]:
image, target = dataset.__getitem__(15)
model.to(image)
results  = model([image])

In [8]:
results

[{'boxes': tensor([[0.0000e+00, 1.5889e+02, 2.7199e+02, 2.7000e+02],
          [2.2726e+02, 1.3758e+00, 4.8000e+02, 1.4613e+02],
          [1.9723e+02, 5.0983e+01, 2.7987e+02, 9.5171e+01],
          [5.6467e+01, 4.7494e+00, 4.8000e+02, 2.5276e+02],
          [1.9249e+02, 6.6411e-01, 2.8090e+02, 2.7264e+01],
          [2.3524e+02, 5.6148e+01, 4.7860e+02, 2.6548e+02],
          [1.9120e+02, 0.0000e+00, 3.2743e+02, 1.2899e+02],
          [1.8587e+02, 4.5876e+01, 2.8909e+02, 1.0824e+02],
          [1.8349e+02, 1.1739e+00, 2.0418e+02, 2.3118e+01],
          [0.0000e+00, 2.0106e+00, 3.6924e+02, 1.9039e+02],
          [2.0320e+02, 1.5344e+01, 2.8811e+02, 1.1801e+02],
          [1.8895e+02, 1.4062e-01, 2.8437e+02, 2.9527e+01],
          [2.5674e+02, 0.0000e+00, 4.0642e+02, 1.2393e+02]],
         grad_fn=<StackBackward0>),
  'labels': tensor([ 1,  1, 14,  1, 14,  1,  1, 17, 17,  1, 14,  1,  1]),
  'scores': tensor([0.9974, 0.9789, 0.9731, 0.2866, 0.2504, 0.1994, 0.1384, 0.1263, 0.1210,
        

In [9]:
torch.save(model.state_dict(), "my_model")

In [ ]:
# class LiveCellDataset(torch.utils.data.Dataset): #torch.utils.data.Dataset
#     def __init__(self, root, transforms):
#         self.root = root
#         self.transforms = transforms
        
#         # Loading annotations here
#         self.coco_annotation = COCO(annotation_file="../data/TrashCan/instances_train_trashcan.json")
        
#         # Loading their ids, please note only one category!
#         self.image_ids = self.coco_annotation.getImgIds(catIds=[self.coco_annotation.getCatIds()[0]])

#     def __getitem__(self, idx):
#         img_id = self.image_ids[idx]
#         img_info = self.coco_annotation.loadImgs([img_id])[0]
#         img_file_name = img_info["file_name"]
#         img_path = Path("../data/TrashCan/train") / img_file_name
#         img = Image.open(str(img_path)).convert("RGB")
#         img = np.asarray(img)
        
        
#         ann_ids = self.coco_annotation.getAnnIds(imgIds=[img_id], iscrowd=None)
#         anns = self.coco_annotation.loadAnns(ann_ids)
        
#         a_mask = self.coco_annotation.annToMask(anns[0])
#         print(a_mask.shape)
        
#         masks, class_ids = self.get_masks(anns, img_info)

# #         print(anns, len(anns))
#         boxes = [a_dict['bbox'] for a_dict in anns]
#         iscrowd = [a_dict['iscrowd'] for a_dict in anns]
#         labels = [a_dict['category_id'] for a_dict in anns]
#         area = [a_dict['area'] for a_dict in anns]
        

#         boxes = torch.as_tensor(boxes, dtype=torch.float32)
#         boxes = box_convert(boxes, in_fmt='xywh', out_fmt='xyxy')
        
#         assert len(boxes) == len(iscrowd) == len(labels) == len(area)
# #         if not (len(boxes) == len(iscrowd) == len(labels) == len(area)):
# #             print(len(boxes), len(iscrowd), len(labels), len(area))

        
#         masks = torch.as_tensor(masks, dtype=torch.uint8)

#         labels = torch.as_tensor(labels, dtype=torch.int64)
#         area = torch.as_tensor(area, dtype=torch.float64)
#         image_id = torch.tensor([idx])
#         iscrowd = torch.as_tensor(iscrowd, dtype=torch.int64)
        
#         target = {}
#         target["boxes"] = boxes
#         target["labels"] = labels
#         target["masks"] = masks
#         target["image_id"] = image_id
#         target["area"] = area
#         target["iscrowd"] = iscrowd
        
        
#         cat_names = np.array(['cell' for x in range(0, len(class_ids))])
#         target["class_ids"] = class_ids
#         target["cat_names"] = cat_names

#         if self.transforms is not None:
#             img, target = self.transforms(img, target)
        
#         return transforms.ToTensor()(img), target
# #         return img, target
    
#     def get_masks(self, annotations, image_info):
#         instance_masks = []
#         class_ids = []
        
#         for annotation in annotations:
# #             print("XD")
#             class_id = annotation['category_id']
#             mask = Image.new('1', (image_info['width'], image_info['height']))
#             mask_draw = ImageDraw.ImageDraw(mask, '1')
#             for segmentation in annotation['segmentation']:
# #                 print("XD2")
#                 mask_draw.polygon(segmentation, fill=1)
#                 bool_array = np.array(mask) > 0
#                 instance_masks.append(bool_array)
#                 class_ids.append(class_id)

#         mask = np.dstack(instance_masks)
# #         print(mask.shape)
#         class_ids = np.array(class_ids, dtype=np.int32)
        
#         return mask, class_ids
        

#     def __len__(self):
#         return len(self.image_ids)